In [61]:
import numpy as np
import numpy as np

x = np.array([
    "0A","1A","2A", # 1
    "0B","1B","2B",
    "0C","1C","2C",
    "0D","1D","2D",
    "3A","4A","5A", # 2
    "3B","4B","5B",
    "3C","4C","5C",
    "3D","4D","5D",
    "6A","7A","8A", # 3
    "6B","7B","8B",
    "6C","7C","8C",
    "6D","7D","8D"
])

B = 4   # batch per sample
M = 3   # number of samples per iteration
N = 3   # number of iterations


y = np.zeros((M*N,B), dtype=x.dtype)


# Equation to map flat index k (0..M*N*B-1) to (row, col) in y:
# row = k % (M*N//B) + (k // (M*N)) * (M*N//B)
# col = (k // (M*N//B)) % B

# Use reshape and transpose to get the desired output for any B, M, N
y[:] = x.reshape(N, B, M).transpose(2, 0, 1).reshape(M*N, B)

# Uninterleave y
def uninterleave(arr, N):
    rows = arr.shape[0]
    M = rows // N
    idx = np.arange(rows)
    inv = (idx % M) * N + (idx // M)
    return arr[inv]

arr = uninterleave(y, N)


#
# We want x in form
#
# [
#   [0A 0B],
#   [1A 1B],
#   [2A 2B],
#   [3A 3B],
#   [4A 4B],
#   [5A 5B],
# ]
#
#
print(y)
print(arr)

[['0A' '0B' '0C' '0D']
 ['3A' '3B' '3C' '3D']
 ['6A' '6B' '6C' '6D']
 ['1A' '1B' '1C' '1D']
 ['4A' '4B' '4C' '4D']
 ['7A' '7B' '7C' '7D']
 ['2A' '2B' '2C' '2D']
 ['5A' '5B' '5C' '5D']
 ['8A' '8B' '8C' '8D']]
[['0A' '0B' '0C' '0D']
 ['1A' '1B' '1C' '1D']
 ['2A' '2B' '2C' '2D']
 ['3A' '3B' '3C' '3D']
 ['4A' '4B' '4C' '4D']
 ['5A' '5B' '5C' '5D']
 ['6A' '6B' '6C' '6D']
 ['7A' '7B' '7C' '7D']
 ['8A' '8B' '8C' '8D']]


In [ ]:

for k in range(M * N * B):
    row = (k % (M * N // B)) + (k // (M * N)) * (M * N // B)
    col = (k // (M * N // B)) % B
    y[row, col] = x[k]


In [ ]:
y[:] = x.reshape(N, B, M).transpose(2, 0, 1).reshape(M*N, B)

#

In [ ]:
# Reshape into (rows, groups)

[['0A' '3A' '6A']
 ['1A' '4A' '7A']
 ['2A' '5A' '8A']
 ['0B' '3B' '6B']
 ['1B' '4B' '7B']
 ['2B' '5B' '8B']
 ['0C' '3C' '6C']
 ['1C' '4C' '7C']
 ['2C' '5C' '8C']]


In [ ]:
# Uninterleave y
def uninterleave(arr, N):
    rows = arr.shape[0]
    M = rows // N  # number of groups
    # forward permutation (interleaved order)
    perm = [i + j*M for i in range(M) for j in range(N)]
    # inverse permutation
    inv = np.argsort(perm)
    return arr[inv]

In [ ]:
y = np.zeros((M*N,B), dtype=x.dtype)


# Equation to map flat index k (0..M*N*B-1) to (row, col) in y:
# row = k % (M*N//B) + (k // (M*N)) * (M*N//B)
# col = (k // (M*N//B)) % B

# Use reshape and transpose to get the desired output for any B, M, N
y[:] = x.reshape(N, B, M).transpose(2, 0, 1).reshape(M*N, B)

# Uninterleave y
def uninterleave(arr, N):
    rows = arr.shape[0]
    M = rows // N  # number of groups
    # forward permutation (interleaved order)
    perm = [i + j*M for i in range(M) for j in range(N)]
    # inverse permutation
    inv = np.argsort(perm)
    return arr[inv]

arr = uninterleave(y, N)

In [ ]:
y = np.zeros((M*N,B), dtype=x.dtype)


# Equation to map flat index k (0..M*N*B-1) to (row, col) in y:
# row = k % (M*N//B) + (k // (M*N)) * (M*N//B)
# col = (k // (M*N//B)) % B

# Use reshape and transpose to get the desired output for any B, M, N
y[:] = x.reshape(N, B, M).transpose(2, 0, 1).reshape(M*N, B)

# Uninterleave y
def uninterleave(arr, N):
    rows = arr.shape[0]
    M = rows // N
    idx = np.arange(rows)
    inv = (idx % M) * N + (idx // M)
    return arr[inv]

In [63]:
import torch

x = np.array(
    [
 [11, 21, 31],
 [12, 22, 32],
 [13, 23, 33],
 [14, 24, 34],

 [41, 51, 61],
 [42, 52, 62],
 [43, 53, 63],
 [44, 54, 64],

 [71, 81, 91],
 [72, 82, 92],
 [73, 83, 93],
 [74, 84, 94]
]
)

x = torch.from_numpy(x)

# Allocate y
y = torch.zeros((M*N, B), dtype=x.dtype, device=x.device)

# Reshape/transpose equivalent
#   x: [M*N, B] → [N, B, M] → [M, N, B] → [M*N, B]
y[:] = x.reshape(N, B, M).permute(2, 0, 1).reshape(M*N, B)

# Uninterleave
def uninterleave(arr, N):
    rows = arr.shape[0]
    M = rows // N
    idx = torch.arange(rows, device=arr.device)
    inv = (idx % M) * N + (idx // M)
    return arr[inv]

arr = uninterleave(y, N)
print(arr)

tensor([[11, 12, 13, 14],
        [21, 22, 23, 24],
        [31, 32, 33, 34],
        [41, 42, 43, 44],
        [51, 52, 53, 54],
        [61, 62, 63, 64],
        [71, 72, 73, 74],
        [81, 82, 83, 84],
        [91, 92, 93, 94]])
